In [1]:
import sys
sys.path.append('..')

import time

import bp3d
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import zarr
from shutil import make_archive, move

In [2]:
client = bp3d.Client(url='https://backburn.sdsc.edu/api') 

In [3]:
lat : 'CWLStringInput' = "39.96263"
lon : 'CWLStringInput' = "-122.60203"
radius : 'CWLIntInput' = 500
sim_time : 'CWLIntInput' = 20
wind_speed : 'CWLIntInput' = 100
wind_direction : 'CWLIntInput' = 100
surface_moisture : 'CWLStringInput' = "0.3"
canopy_moisture : 'CWLStringInput' = "0.3"
steps_fire : 'CWLIntInput' = 10
output_file : 'CWLFilePathOutput' = "./quicfire-output.zip"

In [4]:
# get 1km x 1km domain
fuel = client.fuel(lat=float(lat), lon=float(lon), radius=radius)

In [5]:
# create quicfire run
ens = client.ensemble(fuel=fuel,
                      sim_time=sim_time,
                      wind_speed=wind_speed,
                      wind_direction=wind_direction,
                      surface_moisture=float(surface_moisture),
                      canopy_moisture=float(canopy_moisture),
                      output={
                         'steps_fire':steps_fire
                      }
)

In [6]:
# start execution
ens.execute()

starting {'sim_time': 20, 'fuel': {'id': 'b73571e5-832a-4394-8937-24d9d5446430'}, 'wind_speed': 100, 'wind_direction': 100, 'surface_moisture': 0.3, 'canopy_moisture': 0.3, 'output': {'steps_fire': 10}, 'run_id': 'unknown'}


In [7]:
# save after excution starts to save the run ids
ens.save('quicfire-run.bp3d.json', overwrite=True)

In [8]:
# wait for completion
ens.complete()

all runs failed.


In [9]:
# save outputs
outs = ens.output(ens.df)
z = outs[0].zarr

In [10]:
output_dir = output_file + ".zarr"
store = zarr.NestedDirectoryStore(output_dir)
root = zarr.group(store=store, overwrite=True)
for name,arr in z.arrays():
    root[name] = arr

In [11]:
make_archive(output_file, "zip", output_dir)
move(output_file + ".zip", output_file)

'./quicfire-output.zip'